# Multi Class classification with pytorch using custom neural networks
![](https://drive.google.com/uc?id=1CIA-YaLW29zTSItVYvee81eyvjj6K1U_)

In [ ]:
# Datset link
# https://archive.ics.uci.edu/dataset/53/iris

# We will use this dataset from sklearn
# https://scikit-learn.org/dev/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

In [5]:
X_train, y_train = train_df.drop(columns=['target']), train_df['target']
X_test, y_test = test_df.drop(columns=['target']), test_df['target']

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
class IrisClassifier(nn.Module):
  def __init__(self, input_dim, hidden_dim,output_dim):
    super(IrisClassifier, self).__init__()
    self.network = nn.Sequential(
        nn.Linear(input_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,output_dim),
        nn.Softmax()
    )

  def forward(self, x):
    return self.network(x)


In [10]:
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 3

In [11]:
model = IrisClassifier(input_dim, hidden_dim, output_dim)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [14]:
# Train the model

epochs = 500
for epoch in range(epochs):
  model.train()             #to turn on dropout while training etc
  optimizer.zero_grad()

  predictions = model(X_train_tensor)
  loss = criterion(predictions, y_train_tensor)

  loss.backward()
  optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{epochs}], loss : {loss.item():.4f}")

Epoch [10/500], loss : 0.5989
Epoch [20/500], loss : 0.5852
Epoch [30/500], loss : 0.5791
Epoch [40/500], loss : 0.5754
Epoch [50/500], loss : 0.5718
Epoch [60/500], loss : 0.5698
Epoch [70/500], loss : 0.5683
Epoch [80/500], loss : 0.5673
Epoch [90/500], loss : 0.5662
Epoch [100/500], loss : 0.5654
Epoch [110/500], loss : 0.5646
Epoch [120/500], loss : 0.5640
Epoch [130/500], loss : 0.5634
Epoch [140/500], loss : 0.5629
Epoch [150/500], loss : 0.5625
Epoch [160/500], loss : 0.5622
Epoch [170/500], loss : 0.5619
Epoch [180/500], loss : 0.5616
Epoch [190/500], loss : 0.5614
Epoch [200/500], loss : 0.5613
Epoch [210/500], loss : 0.5611
Epoch [220/500], loss : 0.5610
Epoch [230/500], loss : 0.5609
Epoch [240/500], loss : 0.5608
Epoch [250/500], loss : 0.5607
Epoch [260/500], loss : 0.5606
Epoch [270/500], loss : 0.5605
Epoch [280/500], loss : 0.5605
Epoch [290/500], loss : 0.5604
Epoch [300/500], loss : 0.5604
Epoch [310/500], loss : 0.5603
Epoch [320/500], loss : 0.5603
Epoch [330/500], 

In [17]:
model.eval()           #turn off dropout etc
with torch.no_grad():     #to turn off auto grad as we are only doing prediction
  y_pred = model(X_test_tensor)
  print(y_pred)
  y_pred_labels = torch.argmax(y_pred, dim=1)

  accuracy = (y_pred_labels == y_test_tensor).sum().item() / y_test_tensor.size(0)
  print(f"Test accuracy : {accuracy:.4f}")

tensor([[9.9999e-01, 8.9969e-06, 8.1430e-17],
        [2.2225e-05, 1.3582e-02, 9.8640e-01],
        [2.4923e-05, 9.9997e-01, 2.3196e-07],
        [5.2006e-06, 9.9999e-01, 1.1775e-07],
        [1.0000e+00, 4.3114e-06, 3.1115e-17],
        [1.6610e-04, 9.9980e-01, 3.5391e-05],
        [1.0000e+00, 7.8419e-07, 7.0681e-19],
        [9.9997e-01, 3.4581e-05, 2.5768e-17],
        [1.6435e-11, 1.1169e-10, 1.0000e+00],
        [1.4706e-04, 9.9984e-01, 1.3617e-05],
        [1.4077e-14, 2.3538e-13, 1.0000e+00],
        [4.9096e-16, 5.5364e-17, 1.0000e+00],
        [2.9134e-16, 8.0082e-17, 1.0000e+00],
        [7.1663e-05, 9.9993e-01, 2.8952e-06],
        [9.9999e-01, 6.4884e-06, 1.2124e-17],
        [9.9999e-01, 7.5849e-06, 1.2263e-17],
        [1.0000e+00, 2.5951e-06, 5.2309e-20],
        [1.6611e-06, 1.0000e+00, 5.2954e-08],
        [9.5329e-06, 9.9999e-01, 3.9115e-07],
        [1.1891e-04, 1.1166e-01, 8.8822e-01],
        [1.0000e+00, 3.9777e-06, 1.6705e-18],
        [3.0439e-18, 9.1916e-20, 1

c:\Users\aman0\anaconda3\envs\TorchLearnEnv\Lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [37]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [18]:
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
  input_data = scaler.transform(input_data)
  input_tesnor = torch.tensor(input_data, dtype = torch.float32)

  model.eval()
  with torch.no_grad():
    prediction = model(input_tesnor)
    predicted_class = torch.argmax(prediction, dim=1).item()
  return iris.target_names[predicted_class]

In [19]:
predicted_class = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"Predicted species : {predicted_class}")

Predicted species : setosa


c:\Users\aman0\anaconda3\envs\TorchLearnEnv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\aman0\anaconda3\envs\TorchLearnEnv\Lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
